# PolarNet: An Improved Grid Representation for Online LiDAR Point Clouds Semantic Segmentation

## Paper Reviews

### Prior Reseach

LiDAR를 통해서 얻어지는 Point Cloud를 2D 변환을 통해 3D representation을 얻는 방법은 **front-view** 또는 **bird's-eye-view** 변환이 대표적     
둘은 모두 quantization, projection을 통해서 computationally expensive한 3D operation을 회피한다는 점에서 좋은 선택이었음  
그러나 depth-map을 통해 front-view 변환은 scale, range information을 잃지 않는 BEV보다 empirical하게 performance가 좋지 못했음  

따라서 BEV 변환 후에 좋은 representation을 얻는 것이 중요한데 지금까지의 approach는 **Cartesian coordinate**로 quantization이 이루어짐  
→ 그러나 LiDAR scanner와 가까울수록 많은 point가 scan되는 특성에 따라 Cartesian map은 label과 일치하는 quantization이 어려움  

∴ 조금 더 좋은 quantization/catching representation을 위해서 **polar coordinate** 상에서 convolution을 수행하는 **PolarNet**을 제안    

<p align="center">
<img width="898" alt="1" src="https://user-images.githubusercontent.com/86907286/204104229-eb74197f-e886-403b-9d4c-682d40b7fe9c.png">
</p>



### Polar Bird's-Eye-View

3D representation에서 BEV 변환은 많은 발전이 이루어진 natural image에 대한 CNN을 적용할 수 있는 변환으로서 많이 활용되었음  
단순한 top-down projection을 넘어서 height, reflection 등 많은 것을 동시에 encode하는 접근이 있지만 여전히 **Cartesian coordinate**를 사용하고 있음  
이는 이후 convolution을 위해 quantization을 진행하는 과정에서 **censor가 위치하는 중앙 근처의 grid에 많은 point가 밀집**하는 현상으로 이어짐  
→ computational power를 낭비할 뿐만 아니라 **서로 다른 label을 갖는 point가 섞여** representation을 catch하는데 어려움이 발생할 수 있음 

∴ censor가 위치한 center를 중심으로 LiDAR point가 얻어진다는 점에서 **polar coordinate**를 통한 quantization이 조금 더 자연스러운 접근법이라고 할 수 있음

<p align="center">
<img width="436" alt="2" src="https://user-images.githubusercontent.com/86907286/204104235-5cb991a0-76af-4e19-98a2-554fe59ac32c.png">
</p>

### Learning the Polar Grid

polar coordinate를 기준으로 quantization을 진행한다고 해도 바로 hand-crafted feature로 변환하는 것은 비효율적이라고 할 수 있음  
→ **PointNet**처럼 polar grid(또는 Cartesian grid)를 기준으로 일정 grid 내부에서 shared weight MLP $h(\cdot)$을 통해 representation을 catch하여 사용

$$ \text{fea}_{i, j} = \max (\{h(p) | w_i < p_x < w_{i+1},  l_j < p_y < l_{j+1}\}) $$

이렇게 얻어진 point representation을 polar grid로 나누어서 convolution을 통해 prediction을 위한 grid representation을 얻을 필요가 있음  
이때 polar grid 상에서의 2d convolution을 진행하기 위해서는 **하나의 polar grid cell에서 인접한 모든 cell**을 convolution 시에 참조할 필요가 있음  
→ polar grid 상에서 상하좌우로 연결된 cell을 참조하는(≈ circular convolution) **ring convolution**으로 2d convolution을 대체하여 CNN Architecture 구성

<p align="center">
<img width="436" alt="3" src="https://user-images.githubusercontent.com/86907286/204104238-bd117544-e1a7-4d55-b154-07547328b58c.png">
</p>

### Impact of Projection Method

실제로 이렇게 polar BEV를 기존의 2D spherical projection, Cartesian BEV와 서로 다른 backbone을 사용하여 비교한 결과는 다음과 같음

1. spherical projection은 quantization error와 distance 정보의 상실로 인해 인접한 same label point를 다르게 classification하는 경향이 관찰됨    
2. BEV 방식은 공통적으로 distance가 멀어질 경우 sample이 적어져 mIoU가 비슷해졌지만 인접한 경우에는 Cartesian BEV의 mIoU가 더 낮았음   

<p align="center">
<img width="436" alt="4" src="https://user-images.githubusercontent.com/86907286/204104239-0180ff3e-c769-4686-9248-9d9ed2c353c1.png">
</p>

## Implementation Reviews

Changing Dataset to Polar Grid

In [ ]:
class spherical_dataset(data.Dataset):
  def __init__(self, in_dataset, grid_size, rotate_aug = False, flip_aug = False, ignore_label = 255, return_test = False,
               fixed_volume_space= False, max_volume_space = [50,np.pi,1.5], min_volume_space = [3,-np.pi,-3]):
        'Initialization'
        self.point_cloud_dataset = in_dataset
        self.grid_size = np.asarray(grid_size)
        self.rotate_aug = rotate_aug
        self.flip_aug = flip_aug
        self.ignore_label = ignore_label
        self.return_test = return_test
        self.fixed_volume_space = fixed_volume_space
        self.max_volume_space = max_volume_space
        self.min_volume_space = min_volume_space

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.point_cloud_dataset)

  def __getitem__(self, index):
        'Generates one sample of data'
        data = self.point_cloud_dataset[index]
        if len(data) == 2:
            xyz,labels = data
        elif len(data) == 3:
            xyz,labels,sig = data
            if len(sig.shape) == 2: sig = np.squeeze(sig)
        else: raise Exception('Return invalid data tuple')
        
        # random data augmentation by rotation
        if self.rotate_aug:
            rotate_rad = np.deg2rad(np.random.random()*360)
            c, s = np.cos(rotate_rad), np.sin(rotate_rad)
            j = np.matrix([[c, s], [-s, c]])
            xyz[:,:2] = np.dot( xyz[:,:2],j)

        # random data augmentation by flip x , y or x+y
        if self.flip_aug:
            flip_type = np.random.choice(4,1)
            if flip_type==1:
                xyz[:,0] = -xyz[:,0]
            elif flip_type==2:
                xyz[:,1] = -xyz[:,1]
            elif flip_type==3:
                xyz[:,:2] = -xyz[:,:2]

        # convert coordinate into polar coordinates
        xyz_pol = cart2polar(xyz)
        
        max_bound_r = np.percentile(xyz_pol[:,0],100,axis = 0)
        min_bound_r = np.percentile(xyz_pol[:,0],0,axis = 0)
        max_bound = np.max(xyz_pol[:,1:],axis = 0)
        min_bound = np.min(xyz_pol[:,1:],axis = 0)
        max_bound = np.concatenate(([max_bound_r],max_bound))
        min_bound = np.concatenate(([min_bound_r],min_bound))
        if self.fixed_volume_space:
            max_bound = np.asarray(self.max_volume_space)
            min_bound = np.asarray(self.min_volume_space)

        # get grid index
        crop_range = max_bound - min_bound
        cur_grid_size = self.grid_size
        intervals = crop_range/(cur_grid_size-1)

        if (intervals==0).any(): print("Zero interval!")
        grid_ind = (np.floor((np.clip(xyz_pol,min_bound,max_bound)-min_bound)/intervals)).astype(np.int)

        # process voxel position
        voxel_position = np.zeros(self.grid_size,dtype = np.float32)
        dim_array = np.ones(len(self.grid_size)+1,int)
        dim_array[0] = -1 
        voxel_position = np.indices(self.grid_size)*intervals.reshape(dim_array) + min_bound.reshape(dim_array)
        # voxel_position = polar2cat(voxel_position)
        
        # process labels
        processed_label = np.ones(self.grid_size,dtype = np.uint8)*self.ignore_label
        label_voxel_pair = np.concatenate([grid_ind,labels],axis = 1)
        label_voxel_pair = label_voxel_pair[np.lexsort((grid_ind[:,0],grid_ind[:,1],grid_ind[:,2])),:]
        processed_label = nb_process_label(np.copy(processed_label),label_voxel_pair)
        # data_tuple = (voxel_position,processed_label)

        # prepare visiblity feature
        # find max distance index in each angle,height pair
        valid_label = np.zeros_like(processed_label,dtype=bool)
        valid_label[grid_ind[:,0],grid_ind[:,1],grid_ind[:,2]] = True
        valid_label = valid_label[::-1]
        max_distance_index = np.argmax(valid_label,axis=0)
        max_distance = max_bound[0]-intervals[0]*(max_distance_index)
        distance_feature = np.expand_dims(max_distance, axis=2)-np.transpose(voxel_position[0],(1,2,0))
        distance_feature = np.transpose(distance_feature,(1,2,0))
        # convert to boolean feature
        distance_feature = (distance_feature>0)*-1.
        distance_feature[grid_ind[:,2],grid_ind[:,0],grid_ind[:,1]]=1.

        data_tuple = (distance_feature,processed_label)

        # center data on each voxel for PTnet
        voxel_centers = (grid_ind.astype(np.float32) + 0.5)*intervals + min_bound
        return_xyz = xyz_pol - voxel_centers
        return_xyz = np.concatenate((return_xyz,xyz_pol,xyz[:,:2]),axis = 1)

        if len(data) == 2:
            return_fea = return_xyz
        elif len(data) == 3:
            return_fea = np.concatenate((return_xyz,sig[...,np.newaxis]),axis = 1)
        
        if self.return_test:
            data_tuple += (grid_ind,labels,return_fea,index)
        else:
            data_tuple += (grid_ind,labels,return_fea)
        return data_tuple

Ring Convolution

In [ ]:
class double_conv_circular(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch,group_conv,dilation=1):
        super(double_conv_circular, self).__init__()
        if group_conv:
            self.conv1 = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, padding=(1,0),groups = min(out_ch,in_ch)),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace=True)
            )
            self.conv2 = nn.Sequential(
                nn.Conv2d(out_ch, out_ch, 3, padding=(1,0),groups = out_ch),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace=True)
            )
        else:
            self.conv1 = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, padding=(1,0)),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace=True)
            )
            self.conv2 = nn.Sequential(
                nn.Conv2d(out_ch, out_ch, 3, padding=(1,0)),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace=True)
            )

    def forward(self, x):
        #add circular padding
        x = F.pad(x,(1,1,0,0),mode = 'circular')
        x = self.conv1(x)
        x = F.pad(x,(1,1,0,0),mode = 'circular')
        x = self.conv2(x)
        return x

## Reference

- https://arxiv.org/abs/2003.14032
- https://github.com/edwardzhou130/PolarSeg